In [1]:
import pandas as pd

In [12]:
df = pd.read_csv('rawdata/yearly_data1.csv')

df.head(5)

,ValidDate,ValidTime,AQSID,SiteName,GMTOffset,MeasurementType,MeasurementUnit,RawMeasurement,DataSource
0,01/01/14,00:00,250250002,BOSTON-KENMORE,-5,NO2,PPB,26.0,Massachusetts Dept. of Environmental Protection
1,01/01/14,00:00,250250002,BOSTON-KENMORE,-5,CO,PPM,0.2,Massachusetts Dept. of Environmental Protection
2,01/01/14,00:00,250250002,BOSTON-KENMORE,-5,SO2,PPB,3.0,Massachusetts Dept. of Environmental Protection
3,01/01/14,00:00,250250002,BOSTON-KENMORE,-5,NOX,PPB,39.0,Massachusetts Dept. of Environmental Protection
4,01/01/14,00:00,250250002,BOSTON-KENMORE,-5,NO,PPB,13.0,Massachusetts Dept. of Environmental Protection


In [13]:
df = df.drop(['GMTOffset', 'ValidTime', 'DataSource'], axis=1)

df.head(5)

,ValidDate,AQSID,SiteName,MeasurementType,MeasurementUnit,RawMeasurement
0,01/01/14,250250002,BOSTON-KENMORE,NO2,PPB,26.0
1,01/01/14,250250002,BOSTON-KENMORE,CO,PPM,0.2
2,01/01/14,250250002,BOSTON-KENMORE,SO2,PPB,3.0
3,01/01/14,250250002,BOSTON-KENMORE,NOX,PPB,39.0
4,01/01/14,250250002,BOSTON-KENMORE,NO,PPB,13.0


In [14]:
df = df[(df['MeasurementType'] == 'NO2') | (df['MeasurementType'] == 'PM2.5') | (df['MeasurementType'] == 'OZONE') ]

df.head(5)

,ValidDate,AQSID,SiteName,MeasurementType,MeasurementUnit,RawMeasurement
0,01/01/14,250250002,BOSTON-KENMORE,NO2,PPB,26.0
9,01/01/14,250250041,Boston - Long Island,OZONE,PPB,21.0
10,01/01/14,250250041,Boston - Long Island,NO2,PPB,13.0
15,01/01/14,250250042,Boston - Roxbury,OZONE,PPB,8.0
16,01/01/14,250250042,Boston - Roxbury,NO2,PPB,29.0


In [17]:
df["MeasurementUnit"].value_counts()

MeasurementUnit
PPB      281
UG/M3    174
Name: count, dtype: int64

In [16]:
groups = df.groupby(['ValidDate', 'SiteName'])
print(groups)